In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")



import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re

import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

from tqdm import tqdm
import os

In [2]:
df_text= pd.read_excel(r'D:\Syllabus and guidelines\662- Optimization Methods in ML\Project1\LinkingFileAndText.xlsx')

In [3]:
df_text.head()

,Scroll Number,Image File Name,Text FileName,Text
0,1,04_binladen.jpg,BinLaden.txt,Name: Bin Laden - 11th September\nTranslation ...
1,2,30_theabductionofsita.jpg,TheAbductionOfSita.txt,Name: The Abduction of Sita\nTranslation Sourc...
2,4,08_tsunami.jpg,Tsunami.txt,Name: Tsunami\nTranslation Source: Wesleyan Un...
3,5,17_krishna.jpg,KrishnaLila.txt,Name: Krishna Lila\nTranslation Source: Wesley...
4,6,28_manasamangal.jpg,ManashaMangal.txt,Name: Manasha Mangal\nTranslation Source: Wesl...


In [4]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [5]:
from tqdm import tqdm

preprocessed_reviews = []

for sentence in tqdm(df_text['Text'].values):
    
    
    
    sentence = re.sub('[^A-Za-z]+', ' ', sentence)
    
    sentence = ' '.join(e.lower() for e in sentence.split() if e.lower() not in stopwords)
    
    preprocessed_reviews.append(sentence.strip())

100%|████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 2643.20it/s]


In [6]:
#checking on how does the processed review looks
preprocessed_reviews[28]

'name victimization women translation source wesleyan university artist swarna chitrakar text listen everyone listen carefully listen tale torture dowry demands man named kalipada lived ranaghat nadia district two brothers named kalipada haripada mother name durga rani great one somehow family three pulled along brothers gone school kalipada elder passed got job bardhaman district sometime later came home durga planned arrange marriage heard girl baruipur ushadidi daughter rajen sarkar baruipur beautiful fair post graduate kalipada saw approved haggling dowry rajen give rs cash watch gold chain earrings bangles agreement couple went spend wedding night morning rajen came short rs hearing kalipada family started abusing usha mortally hurt thought show face night anguish hanged listen inhabitants india morning neighbors came bewildered rajen came hearing news daughter kill mother law said know anything daughter law kill others said know durga rani angry dowry short know let go police sta

In [7]:
# Featurization
count_vect = CountVectorizer() #in scikit-learn
count_vect.fit(preprocessed_reviews)
print("showcasing some feature", count_vect.get_feature_names()[:10])
print('-'*50)

final_counts = count_vect.transform(preprocessed_reviews)
print("the type of count vectorizer ",type(final_counts))
print("the shape of out text BOW vectorizer ",final_counts.get_shape())
print("the number of unique words ", final_counts.get_shape()[1])

showcasing some feature ['abandoned', 'abduction', 'able', 'abroad', 'abu', 'abuse', 'abused', 'abusing', 'access', 'accident']
--------------------------------------------------
the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text BOW vectorizer  (29, 1231)
the number of unique words  1231


In [8]:
df_final_text= pd.DataFrame()
df_final_text['FileName']= df_text['Image File Name']


In [9]:
listFinal= list(final_counts.toarray())

column_list=[]
for k in range(1,(final_counts.get_shape()[1]) +1):
  col_name= "f"+ str(k)
  column_list.append(col_name)

print(len(column_list))

#defining a new features dataframe with each of the 1231 features
#of each file as rows
df_text_features= pd.DataFrame(listFinal, columns=column_list )

#lets check out what we got
df_text_features.head()

1231


,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f1222,f1223,f1224,f1225,f1226,f1227,f1228,f1229,f1230,f1231
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1


In [10]:
df_concat = pd.concat([df_final_text, df_text_features], axis=1)
df_concat.head()

,FileName,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f1222,f1223,f1224,f1225,f1226,f1227,f1228,f1229,f1230,f1231
0,04_binladen.jpg,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,30_theabductionofsita.jpg,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,08_tsunami.jpg,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,17_krishna.jpg,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,28_manasamangal.jpg,0,0,0,0,0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,1


In [11]:
df_concat.shape

(29, 1232)

In [12]:
#appending labels to the extracted tree text features
df= pd.read_excel(r'D:\Syllabus and guidelines\662- Optimization Methods in ML\Project1\train_tree\tree_text_and_labels.xlsx')
df.head()


,Scroll Number,Image File Name,Text FileName,Text,Labels
0,1,04_binladen.jpg,BinLaden.txt,Name: Bin Laden - 11th September\nTranslation ...,1
1,2,30_theabductionofsita.jpg,TheAbductionOfSita.txt,Name: The Abduction of Sita\nTranslation Sourc...,1
2,4,08_tsunami.jpg,Tsunami.txt,Name: Tsunami\nTranslation Source: Wesleyan Un...,0
3,5,17_krishna.jpg,KrishnaLila.txt,Name: Krishna Lila\nTranslation Source: Wesley...,1
4,6,28_manasamangal.jpg,ManashaMangal.txt,Name: Manasha Mangal\nTranslation Source: Wesl...,0


In [13]:
list_labels= list(df['Labels'])


In [14]:
df_concat.insert(loc=1232, column= "Class_label", value= list_labels )

df_concat.tail()

,FileName,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f1223,f1224,f1225,f1226,f1227,f1228,f1229,f1230,f1231,Class_label
24,33_binladen11september.jpg,0,0,1,0,0,0,0,0,0,...,0,0,0,0,3,0,0,0,0,0
25,21_girlchild.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26,07_gujarat.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,18_tsunami.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28,12_victimizationofwomen.jpg,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df_tree_text_featuresandlabels= df_concat

df_tree_text_featuresandlabels.to_excel(r'D:\Syllabus and guidelines\662- Optimization Methods in ML\Project1\train_tree\tree_text_featuresand_labels.xlsx', index= False)

# For Mythchars

In [16]:
del df_concat['Class_label']
df= pd.read_excel(r'D:\Syllabus and guidelines\662- Optimization Methods in ML\Project1\train_mythchar\mythchar_text_and_labels.xlsx')

list_labels= list(df['Labels'])

df_concat.insert(loc=1232, column= "Class_label", value= list_labels )

df_concat.tail()

,FileName,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f1223,f1224,f1225,f1226,f1227,f1228,f1229,f1230,f1231,Class_label
24,33_binladen11september.jpg,0,0,1,0,0,0,0,0,0,...,0,0,0,0,3,0,0,0,0,0
25,21_girlchild.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26,07_gujarat.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,18_tsunami.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
28,12_victimizationofwomen.jpg,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df_mythchar_text_featuresandlabels= df_concat

df_mythchar_text_featuresandlabels.to_excel(r'D:\Syllabus and guidelines\662- Optimization Methods in ML\Project1\train_mythchar\mythchar_text_featuresand_labels.xlsx', index= False)

In [18]:
del df_concat['Class_label']
df= pd.read_excel(r'D:\Syllabus and guidelines\662- Optimization Methods in ML\Project1\train_animal\animal_text_and_labels.xlsx')

list_labels= list(df['Labels'])

df_concat.insert(loc=1232, column= "Class_label", value= list_labels )

df_concat.tail()

,FileName,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f1223,f1224,f1225,f1226,f1227,f1228,f1229,f1230,f1231,Class_label
24,33_binladen11september.jpg,0,0,1,0,0,0,0,0,0,...,0,0,0,0,3,0,0,0,0,0
25,21_girlchild.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26,07_gujarat.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,18_tsunami.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28,12_victimizationofwomen.jpg,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
df_animal_text_featuresandlabels= df_concat

df_animal_text_featuresandlabels.to_excel(r'D:\Syllabus and guidelines\662- Optimization Methods in ML\Project1\train_animal\animal_text_featuresand_labels.xlsx', index= False)